In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.naive_bayes
import sklearn.ensemble as ske
import sklearn.neighbors
import sklearn.pipeline
import sklearn.linear_model
import joblib
from sklearn import preprocessing
from sklearn import tree, linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel
import sys
import sqlite3
import importlib

In [2]:
df_wild = pd.read_pickle('/content/drive/MyDrive/học tập/DACN/wild.pickle')

In [18]:
print(df_wild.shape)

(50724, 56555)


In [19]:
df_wild.head()

,sample_sha1,benign,malicious,packed,unpacked_sample_sha1,unpacked_sample_id,packer_name,imp_setupremovefilelogentrya,imp_vardeccmp,imp_setdlgitemint,...,opcode_xor sub mov,opcode_xor sub mov add,opcode_xor test,opcode_xor test je,opcode_xor xor,opcode_xor xor mov,opcode_xor xor pop,opcode_xor xor pop mov,opcode_xor xor xor,opcode_xor xor xor pop
1,a2aac23e17be570c647fd22080e05b0e58449565,True,False,True,-1,-1,none,False,False,False,...,0.000022,0.0,0.000088,0.000084,0.000070,0.000066,0.000028,0.0,0.000053,0.0
2,c3aa534bbd20f3a3dd8e7a457f625291af9d0fe4,False,True,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000309,0.000000,0.000000,0.0,0.000331,0.0
3,bea3fb5a68ac91258e8339a8aa037304ae24242b,True,False,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
10,15cb9eab6b0f0ccd718a9235f07046dff100789f,False,True,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
12,8792cf08950abada1572b47948e86c9fce26c01a,True,False,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


Chia nhỏ dataset

In [3]:
#Chọn ngẫu nhiên 12,000 mẫu
df = df_wild.sample(n=12000, random_state=102)

Sử lý dataset

In [4]:
# Loại bỏ các lables không cần thiết
df = df.drop(['sample_sha1','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
df = df.drop(['benign_vt','malicious_vt'], axis=1)
df = df.drop(['packer_name','source','similarity','most_similar_sha1','unpacked_similarity'], axis=1)


In [5]:
#Xử lý các cột có dạng object
for col in df.columns:
  dtype = df[col].dtype
  if dtype == object:
    le = preprocessing.LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [ ]:
df.head()

,benign,malicious,packed,imp_setupremovefilelogentrya,imp_vardeccmp,imp_setdlgitemint,imp_?append@?$basic_string@du?$char_traits@d@std@@v?$allocator@d@2@@std@@qaeaav12@abv12@ii@z,imp_zwwritefile,imp_glstencilop,imp_comparelinkinforeferents,...,opcode_xor sub mov,opcode_xor sub mov add,opcode_xor test,opcode_xor test je,opcode_xor xor,opcode_xor xor mov,opcode_xor xor pop,opcode_xor xor pop mov,opcode_xor xor xor,opcode_xor xor xor pop
167256,False,True,True,False,False,False,False,False,False,False,...,0.000087,0.0,0.000329,0.000267,0.000086,0.00000,0.0,0.0,0.000000,0.0
14422,False,True,True,False,False,False,False,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0
185351,False,True,True,False,False,False,False,False,False,False,...,0.000029,0.0,0.000105,0.000099,0.000085,0.00011,0.0,0.0,0.000043,0.0
53241,True,False,True,False,False,False,False,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0
38618,True,False,True,False,False,False,False,False,False,False,...,0.000000,0.0,0.000199,0.000124,0.000000,0.00000,0.0,0.0,0.000000,0.0


In [ ]:
print(df.shape)

(12000, 56545)


Chia dataset thành 10000 tập để train và 2000 mẫu để test

In [6]:
df_train = df.sample(n=10000, random_state=102)
df_test = df.drop(df_train.index)

In [ ]:
#Vì tập dữ liệu không có Unpacked Malware, chỉ cần loại bỏ Packed Benign
data = df_train.drop(df_train[(df_train['malicious'] == False) & (df_train['packed'] == True)].index)

KeyError: ignored

In [7]:
#Kiểm tra số lượng mẫu
Unpacked_Benign = df_train[(df_train['benign'] == True) & (df_train['packed'] == False)].shape[0]
Packed_Malware = df_train[(df_train['malicious'] == True) & (df_train['packed'] == True)].shape[0]
print('Unpacked Benign: %i samples' % Unpacked_Benign)
print('Packed Malware: %i samples' % Packed_Malware)
print('Total TRAIN set: %i samples'% (Unpacked_Benign+Packed_Malware))


Unpacked Benign: 851 samples
Packed Malware: 6614 samples
Total TRAIN set: 7465 samples


In [8]:
#Chia tỉ lệ 50 - 50
data_PM = df_train[(df_train['malicious'] == True) & (df_train['packed'] == True)].sample(n=Packed_Malware-Unpacked_Benign, random_state=102)
df_train = df_train.drop(data_PM.index)

In [9]:
#Số lượng tập train
Unpacked_Benign = df_train[(df_train['benign'] == True) & (df_train['packed'] == False)].shape[0]
Packed_Malware = df_train[(df_train['malicious'] == True) & (df_train['packed'] == True)].shape[0]
print('Unpacked Benign: %i samples' % Unpacked_Benign)
print('Packed Malware: %i samples' % Packed_Malware)
print('Total TRAIN set: %i samples'% (Unpacked_Benign+Packed_Malware))

Unpacked Benign: 851 samples
Packed Malware: 851 samples
Total TRAIN set: 1702 samples


In [10]:
#Xử lý tập train
y = df_train['malicious'].values
df_train = df_train.drop(['malicious','packed'], axis=1)

Xử lý tập test

In [12]:
Unpacked_Benign = df_test[(df_test['benign'] == True) & (df_test['packed'] == False)].shape[0]
Packed_Malware = df_test[(df_test['malicious'] == True) & (df_test['packed'] == True)].shape[0]
Packed_Benign = df_test[(df_test['benign'] == True) & (df_test['packed'] == True)].shape[0]

df_test_PM = df_test[(df_test['malicious'] == True) & (df_test['packed'] == True)].sample(n=(Packed_Malware-(Unpacked_Benign+Packed_Benign)), random_state=102)
df_test = df_test.drop(df_test_PM.index)

Unpacked_Benign = df_test[(df_test['benign'] == True) & (df_test['packed'] == False)].shape[0]
Packed_Malware = df_test[(df_test['malicious'] == True) & (df_test['packed'] == True)].shape[0]
Packed_Benign = df_test[(df_test['benign'] == True) & (df_test['packed'] == True)].shape[0]

print('Unpacked Benign: %i samples' % Unpacked_Benign)
print('Packed Malware: %i samples' % Packed_Malware)
print('Packed Benign: %i samples' % Packed_Benign)
print('Total TEST set: %i samples'% (Unpacked_Benign+Packed_Malware+Packed_Benign))

Unpacked Benign: 163 samples
Packed Malware: 680 samples
Packed Benign: 517 samples
Total TEST set: 1360 samples


In [15]:
#Chọn lọc features
print('Researching important feature based on %i total features' % df_train.shape[1])
fsel = ske.ExtraTreesClassifier().fit(df_train, y)

Researching important feature based on 56543 total features


In [16]:
#Tạo mô hình và chia tập train - test
model = SelectFromModel(fsel, prefit=True)

X_train = model.transform(df_train)
X_test = model.transform(df_test.drop(['malicious','packed'], axis=1))
y_train = y
y_test = df_test['malicious'].values

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [17]:
#In ra số lượng features chọn được
nb_features = X_train.shape[1]
features = []
print('%i features identified as important:' % nb_features)

indices = np.argsort(fsel.feature_importances_)[::-1][:nb_features]
print(indices)

6759 features identified as important:
[    0 42160 42708 ... 32139 10809 54477]


In [20]:
#In ra chỉ số quan trọng của từng features
for f in range(nb_features):
    print("%d. feature %s (%f)" % (f + 1, df_train.columns[indices[f]], fsel.feature_importances_[indices[f]]))

#Gán vào tập features để xử lý mô hình
for f in sorted(np.argsort(fsel.feature_importances_)[::-1][:nb_features]):
    features.append(df_train.columns[f])

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
1760. feature opcode_push pop pop (0.000099)
1761. feature opcode_movzx movzx (0.000099)
1762. feature string_b'HttpOpenRequestA' (0.000099)
1763. feature opcode_mov ret mov (0.000099)
1764. feature opcode_call push (0.000099)
1765. feature opcode_call int3 mov push (0.000099)
1766. feature ngram_b'_hypot' (0.000099)
1767. feature opcode_jl (0.000099)
1768. feature imp_lstrcpya (0.000099)
1769. feature ngram_b'e32.dl' (0.000099)
1770. feature rich_pid_0 (0.000099)
1771. feature opcode_shr push push (0.000099)
1772. feature string_b'comctl32.dll' (0.000099)
1773. feature ngram_b'="1.0"' (0.000099)
1774. feature opcode_jae push (0.000099)
1775. feature ngram_b'\x00E\x00x\x00t' (0.000098)
1776. feature ngram_b'\x00t\x00e\x00\x00' (0.000098)
1777. feature string_b'PVVRV' (0.000098)
1778. feature opcode_mov jmp cmp (0.000098)
1779. feature opcode_mov call mov push (0.000098)
1780. feature string_b'D$$Pj' (0.000098)
1781. feature opcod

In [21]:
#Chọn 4 thuật toán
algorithms = {
        "DecisionTree": tree.DecisionTreeClassifier(max_depth=20),
        "RandomForest": ske.RandomForestClassifier(n_estimators=100),
        "GradientBoosting": ske.GradientBoostingClassifier(n_estimators=100),
        "AdaBoost": ske.AdaBoostClassifier(n_estimators=200)
    }
results = {}

#Train - test mô hình với lần lượt 4 thuật toán
print("Now testing algorithms")
for algo in algorithms:
    clf = algorithms[algo]
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("%s : %f %%" % (algo, score*100))
    results[algo] = score

Now testing algorithms
DecisionTree : 100.000000 %
RandomForest : 93.382353 %
GradientBoosting : 100.000000 %
AdaBoost : 100.000000 %


In [22]:
#Chọn mô hình có Acc lớn nhất
winner = max(results, key=results.get)
print('Algorithm with highest accuracy on train/test is %s with a %f %% success' % (winner, results[winner]*100))

Algorithm with highest accuracy on train/test is DecisionTree with a 100.000000 % success


In [23]:
#In ra các chỉ số đánh giá khác của mô hình
clf = algorithms[winner]
res = clf.predict(X_test)
mt = confusion_matrix(y_test, res)

print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))

print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))

True positive : 680 
True negative : 680 
False positive : 0 
False negative : 0 


In [24]:
report = classification_report(y_test, res)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

       False       1.00      1.00      1.00       680
        True       1.00      1.00      1.00       680

    accuracy                           1.00      1360
   macro avg       1.00      1.00      1.00      1360
weighted avg       1.00      1.00      1.00      1360

